In [2]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
# import os
#import string

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

import pickle

In [3]:
df = pd.read_csv('job7_df_all.csv', encoding='utf-8-sig')

In [4]:
# df = df.dropna().reset_index(drop=True)
df.shape

(24096, 771)

In [5]:
# df.head()

# TextCNN 預處理

In [6]:
df_cnn = df.iloc[:,705:-1].copy()
df_cnn.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c55,c56,c57,c58,c59,c60,c61,c62,c63,c64
0,149,465,29,1,64,84,354,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,133,244,119,462,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88,29,395,206,187,418,248,226,5,439,...,0,0,0,0,0,0,0,0,0,0
3,149,353,88,476,36,248,110,5,106,0,...,0,0,0,0,0,0,0,0,0,0
4,149,465,409,110,476,374,248,110,5,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# with open('vocab.pickle', 'rb') as handle:
    # vocab = pickle.load(handle)
    
# num_words = len(vocab)

In [7]:
num_words = 613
max_train_sentence_length = 65

In [22]:
with open('embedding_matrix15.pkl','rb') as f:
    embedding_matrix = pickle.load(f)

# ANN 預處理

In [9]:
df.iloc[:,194:-77].columns

Index(['3G', '3ds-Max', '3ds-Max-Design', '6-Sigma', 'A+', 'ABAQUS', 'ADA',
       'ADC', 'ADO', 'ADSL',
       ...
       '英文打字150以上', '英文打字20~50', '英文打字20以下', '英文打字50~75', '英文打字75~100',
       '金旭飯店管裡系統', '金蝶', '鉅茂', '鼎基-ERP', '鼎新'],
      dtype='object', length=500)

In [10]:
df.iloc[:,-77:-74].columns

Index(['上班時段_label', '縣市_label', '職位類別_label'], dtype='object')

In [11]:
# get dummies

df_pst_ = pd.get_dummies(df['職位_'])
df_pst_cat = pd.get_dummies(df['職位類別'])
df_country = pd.get_dummies(df['縣市'])
df_area = pd.get_dummies(df['地區'])
df_time = pd.get_dummies(df['上班時段'])

df_edu_ = pd.get_dummies(df['學歷要求_'])
df_res_ = pd.get_dummies(df['管理責任_'])
df_dem_ = pd.get_dummies(df['需求人數_'])
df_work_ = pd.get_dummies(df['工作經歷'])

#擅長工具
df_tools = (df.iloc[:,193:-77]).copy()
df_newlabel = (df.iloc[:,-77:-74]).copy()
df = df.astype({'供需人數':'int64'})

In [12]:
# 篩選擅長工具
top_tools = ['JavaScript', 'Linux', 'C#', 'MS-SQL', 'Java', 'Python', 'C++', 'HTML', 'C', 'MySQL', 'ASP.NET', 'CSS', 'jQuery', 'Git', 'Excel', 'Word', 'PHP', 'PowerPoint', 'Oracle', 'TCP/IP', 'VueJS', 'Android', 'AWS', 'Node.js', 'Vmware', 'AJAX', 'Outlook', 'Windows-Server-2012', 'Spring', 'Windows-10', 'Firewall', 'ReactJS', 'iOS', 'Github', 'PostgreSQL', 'Visual-Studio', 'PL/SQL', 'DNS', 'Visual-Studio-.net', 'Windows-Server-2019', 'Shell', 'SWIFT', 'Go', 'Cisco', 'Visual-C#', '鼎新', 'HTTP', 'JSP', 'Visual-Basic-.net', 'Angular', 'Windows-Server-2008', 'Windows-7', 'MCU', 'Objective-C', 'Kotlin', 'Unity3D', 'XML', 'VPN', 'LAN', 'SAP', 'AngularJS', 'UNIX', 'DHCP', 'Matlab', 'C++.Net', 'Tomcat', 'IIS', 'Visual-C++', 'ARM', 'OOP', 'Power-BI', 'Visual-Basic', 'Jenkins', 'VLAN', 'ETL', 'Windows-XP', 'Ethernet', 'J2EE', 'WinForm', 'UML', 'Mac-OS', 'Windows-8', 'RDBMS', 'FTP', 'Django', 'Adobe-Photoshop', 'PLC', 'Sass', 'Socket', 'ReactNative', 'Tableau', 'Security', 'Microsoft-Exchange', 'R', 'ASP', 'Illustrator', '中文打字20~50', 'Struts', 'AutoCAD', 'Routers', 'Firmware', 'WAN', '中文打字50~75', 'Project', 'hadoop', 'LabVIEW', 'Google-Analytics', 'VBA', 'MES', 'DHTML', 'JDBC', 'Visio', 'Assembly', 'XML-Web-services', '英文打字20~50', 'Windows-2000', 'Windows-2003', 'Access', 'Systems-Analysis', 'Database-Management', 'ANSI-SQL', 'Delphi', 'Oracle-ERP', 'FPGA', 'ssh', 'UDP', 'Ruby', 'Verilog', 'Perl', 'VBScript', 'Redux', 'Flutter', 'Internet-Explorer', 'Load-Balancing', 'Windows-NT', 'Web-Master/Developer', 'COBOL', 'DB2', 'LDAP', 'MFC', 'Adobe-Acrobat', 'Database-Administrator', '英文打字50~75', 'SAN/NAS', 'Servlets', 'Juniper', 'Citrix', 'AS/400', 'Unreal-Engine', 'Adobe-XD', 'SAN', 'DSP', 'Microsoft-SharePoint', 'ODBC', 'OOAD', 'Informatica', 'XHTML', 'SAS', 'Sketch', 'Systems-Administration', 'VM', 'IPS', 'Firebase', 'Hubs/-Routers', 'Rails', 'Data-Modeling', 'TcpDump', 'Sniffer', 'WLAN', 'Bluetooth', 'Informix', 'OrCAD', 'Apache-SOAP', 'IDS', 'RTL', 'Windows-98', 'Axure-RP', 'ADO', 'SolidWorks', 'AIX', 'Apple', 'J2SE', 'Scala', 'Hive', 'JSF', 'GIS', 'RPG', 'WebSphere', 'TCL', 'EDA', 'PowerBuilder', 'FreeBSD', 'ActiveX', 'Lotus-Notes', 'iptables', 'Google-Tag-Manager', 'OSPF', 'Win32', 'e-commerce', 'DataStage', 'Domino', 'MPLS', 'OLAP', 'SNMP', 'VHDL', 'CIM', 'Mac/Macintosh', 'SPSS', 'Oracle-Forms', 'Systems-Analyst', 'Teradata', 'BGP', 'Data-Architect', 'Dreamweaver', 'WebLogic', 'Figma', '正航', 'Checkpoint', 'MAYA', 'OS-X', 'SDLC', 'Ghost', '3ds-Max', 'Google-Data-Studio', 'Solaris', 'Zeplin', 'Drivers', 'DOS', 'SAPDB', 'SOAP', 'VoIP', 'Windows-95', '文中系統', 'D3.js', 'Junit', 'Xmind', 'Robot', 'WAP', 'Windows-Vista', 'PADS', 'FrontPage', 'MQSeries', 'VSAM', '英文打字75~100', 'Cognos', 'Dart', 'Flex', 'USB技術', 'WebAssembly', 'ArcGis', 'Data-Marts', 'Google-Trend', 'PPPoE', 'Fortify', 'MRP', 'Sun-Solaris', 'Cadence-Allegro', 'LotusScript', 'Protel', 'Developer/-Designer-2000', 'STL', 'HP-UX', 'Squid', 'After-Effects', 'AutoCad-2D', 'Dbase', '中文打字75~100', 'Circuit-Design', 'Mantis', 'ASIC', 'EJB', 'Sybase', 'VMS', 'Version-Control', 'ADC', 'Adobe-InDesign', 'Blender', 'CGI', 'Toad', 'XSL', '天心資訊', 'DirectX', 'Flash', 'Intrusion', 'OS/400', 'Premiere', 'Sketch-up', 'AutoCad-3D', 'LibreOffice-Writer', '德安飯店餐飲管理系統', 'Bugzilla', 'COM/DCOM', 'PBX', 'RF', 'Test-Scripts', 'Visual-Foxpro', 'WIN-CE', '中文打字20以下', 'Jasmine', 'Publisher', '德安資訊ERP', '鼎基-ERP', 'EDI', 'Fox-Pro', 'InVision', 'Navision', 'Pro/E', '英文打字20以下', 'AdvanceLink', 'CPLD', 'Data-Guard', 'Fireworks', 'SPC', 'Avaya', 'C++test', 'CAM', 'Draw', 'IMS', 'J2ME', 'Microsoft-Dynamics-AX', 'Network-Cards', 'SUN-OS', 'Screaming-Frog-SEO-Spider', 'Silverlight', 'VERITAS', 'ADSL', 'Base', 'Bridges', 'CorelDraw', 'Shtml', 'CC-Mail', 'Electronic-Payment', 'Games', 'Graphics', 'Hubs', 'JCL', 'Pro*C', 'Progress', '英文打字100~125', '鉅茂', 'Alexa', 'Calc', 'Google-Display-Network', 'Impress', 'MicroStrategy', 'OS/390', 'OneNote', 'RIP', 'ActionScript', 'BS7799', 'Clipper', 'GPS全球定位系統', 'IE工業工程', 'LoadRunner', 'PhotoImpact', 'SQR', '金蝶', 'CA', 'Mainframe', 'Microsoft-Photo-Editor', 'Quark-Express', 'Adabas', 'CASE', 'Cinema-4D', 'GSM', 'JMS', 'PowerPCB', 'SMT', 'Wordperfect', '用友U8', '6-Sigma', 'CICS', 'CodeTest', 'FrameMaker', 'ISDN', 'Inventor', 'LanManager', 'P-CAD', 'Pagemaker', 'Planner', 'SYSBASE', 'SimilarWeb', 'WML', 'Windows-Mobile', 'XSLT', '中文打字100~125', 'A+', 'FoxBASE+', 'GPRS', 'ISAPI', 'ISO-9000', 'Motion-Builder', 'Netbios', 'QTP', 'RTSP', 'SuperGIS', 'TIBCO', 'TS16949', 'Visual-J#', 'WebMethods', '上華ERP系統', '浪潮', '金旭飯店管裡系統', 'CVS', 'Catia', 'DDK', 'ERwin', 'FORTRAN', 'FoxPro-2', 'Hyperion-(Brio)', 'ISO-14000', 'Ingres', 'Macromedia-Director', 'Oracle-Financials', 'RoHS', 'Sonet', 'Sublime', 'Baan', 'Banyan', 'CDMA', 'CORBA', 'DVB數位視頻廣播', 'EMC/EMI', 'MIDI', 'MMS', 'MapGIS', 'Mobile-phone', 'Multimedia-Builder', 'NDS/Novell-Directory-Services', 'OS/2', 'OmniGraffle', 'PC—lint', 'PL/1', 'SPICE', 'Scribus', 'Synopsys', 'Telecom', 'TestBed', 'V-Ray', 'WPS', 'X++', 'X.25', 'Zbrush', '中文打字125~150', '中文打字150以上', '3ds-Max-Design', 'ABAQUS', 'ATL', 'Adobe-Animate', 'BizTalk', 'ClearCase', 'ClearQuest', 'Cold-Fusion', 'DEC/VAX', 'DVR數位視頻錄像', 'FileNet', 'Focus', 'HP-Open-View', 'IATF16949', 'LanServer', 'Lantastic', 'Lease-Lines', 'Math', 'Microsoft-SmartPhone', 'NetWare', 'PABX', 'PCBA', 'PSTN', 'QAD－MFG/PRO', 'Revit', 'SMS', 'SNA', 'Silverstream', 'Softimage', 'USB-OTG', 'Visual-SourceSafe', 'VxWorks', '英文打字125~150', '英文打字150以上', 'ADA', 'ArcView', 'Authorware', 'CADAM', 'CoolDraw', 'ISO-45001', 'MVS', 'Mindnode', 'Mobile-Network', 'PDA/Handhelds', 'PTC-Creo-Elements/Direct', 'RMI', 'Rational-Robot', 'Rational-Test-RealTime', 'Rexx', 'Rhino', 'SOLIDWORKS-Electrical', 'TK', 'Tandem', 'Unigraphics', '3G']

# choose top tools
df_tools = df_tools.loc[:, top_tools[:50]].reset_index(drop=True)
df_tools.columns

Index(['JavaScript', 'Linux', 'C#', 'MS-SQL', 'Java', 'Python', 'C++', 'HTML',
       'C', 'MySQL', 'ASP.NET', 'CSS', 'jQuery', 'Git', 'Excel', 'Word', 'PHP',
       'PowerPoint', 'Oracle', 'TCP/IP', 'VueJS', 'Android', 'AWS', 'Node.js',
       'Vmware', 'AJAX', 'Outlook', 'Windows-Server-2012', 'Spring',
       'Windows-10', 'Firewall', 'ReactJS', 'iOS', 'Github', 'PostgreSQL',
       'Visual-Studio', 'PL/SQL', 'DNS', 'Visual-Studio-.net',
       'Windows-Server-2019', 'Shell', 'SWIFT', 'Go', 'Cisco', 'Visual-C#',
       '鼎新', 'HTTP', 'JSP', 'Visual-Basic-.net', 'Angular'],
      dtype='object')

In [13]:
# label
col_list = ['職位類別_label', '縣市_label', '上班時段_label', '外商', '供需人數']
df_ = df.loc[:, col_list].reset_index(drop=True)

df_label = pd.concat([df_, df_pst_, df_area, df_edu_, df_res_, df_dem_, df_work_, df_tools, df_newlabel], axis=1)


# dummies
col_list2 = ['外商', '供需人數']
df2_ = df.loc[:, col_list2].reset_index(drop=True)


df_dummies = pd.concat([df2_, df_pst_cat, df_pst_, df_country, df_time,
                        df_edu_, df_res_, df_dem_, df_work_, df_tools, df_newlabel], axis=1)

In [14]:
# 兩種版本
# df_ann = df_label.copy()
df_ann = df_dummies.copy()

# Split Training/ Test

In [15]:
# type(df[['salary_avg']])

In [16]:
df_concat = pd.concat([df_cnn, df_ann, df[['salary_avg']]], axis=1)
df_concat = shuffle(df_concat, random_state = 28374)
salary = df_concat['salary_avg'].tolist()
df_concat = df_concat.drop('salary_avg', axis=1)

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(df_concat,\
                                                      salary,\
                                                      test_size = 0.1,  
                                                      random_state = 887762)

In [18]:
print('x_train shape', X_train.shape)
print('x_valid shape', X_valid.shape)

x_train shape (21686, 164)
x_valid shape (2410, 164)


In [19]:
# cnn
X_train_cnn = X_train.iloc[:, :65]
X_valid_cnn = X_valid.iloc[:, :65]
# X_train_cnn = X_train_cnn.values.tolist()
# X_valid_cnn = X_valid_cnn.values.tolist()
# X_train_cnn = np.asarray(X_train_cnn)
# X_valid_cnn = np.asarray(X_valid_cnn)
print('X_train_cnn shape', X_train_cnn.shape)
print('X_valid_cnn shape', X_valid_cnn.shape)

# ann
X_train_ann = X_train.iloc[:, 65:]
X_valid_ann = X_valid.iloc[:, 65:]
# X_train_ann = np.asarray(X_train_ann)
# X_valid_ann = np.asarray(X_valid_ann)
print('X_train_ann shape', X_train_ann.shape)
print('X_valid_ann shape', X_valid_ann.shape)

# train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train))
# valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test))

y_train = np.asarray(y_train)
y_valid = np.asarray(y_valid)
print('y_train shape', y_train.shape)
print('y_valid shape', y_valid.shape)

X_train_cnn shape (21686, 65)
X_valid_cnn shape (2410, 65)
X_train_ann shape (21686, 99)
X_valid_ann shape (2410, 99)
y_train shape (21686,)
y_valid shape (2410,)


# Model

In [23]:
def model_(X_train_cnn, X_valid_cnn, X_train_ann, X_valid_ann, y_train, y_valid, embedding_matrix):

    # 構建TextCNN模型
    cnn_input = tf.keras.layers.Input(shape=(max_train_sentence_length,), dtype='float64')

    # embedding
    embedder = tf.keras.layers.Embedding(num_words + 1, 15, input_length=max_train_sentence_length, 
                                            weights=[embedding_matrix], trainable=False)
    embed = embedder(cnn_input)

    # cnn1
    cnn1 = tf.keras.layers.Conv1D(256, 3, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(embed)

    cnn1 = tf.keras.layers.Conv1D(256, 3, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn1)
    #max pooling                         
    # cnn1 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn1)

    cnn1 = tf.keras.layers.Conv1D(128, 3, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(cnn1)

    cnn1 = tf.keras.layers.Conv1D(128, 3, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn1)
    #max pooling                         
    # cnn1 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn1)

    ############################################

    ## conv aize = 4
    cnn2 = tf.keras.layers.Conv1D(256, 4, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(embed)

    cnn2 = tf.keras.layers.Conv1D(256, 4, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn2)
    #max pooling                         
    # cnn2 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn2)

    cnn2 = tf.keras.layers.Conv1D(128, 4, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(cnn2)

    cnn2 = tf.keras.layers.Conv1D(128, 4, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn2)
    #max pooling                         
    # cnn2 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn2)

    ############################################


    ## conv aize = 5
    cnn3 = tf.keras.layers.Conv1D(256, 5, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(embed)

    cnn3 = tf.keras.layers.Conv1D(256, 5, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn3)
    #max pooling                         
    # cnn3 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn3)

    cnn3 = tf.keras.layers.Conv1D(128, 5, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(cnn3)

    cnn3 = tf.keras.layers.Conv1D(128, 5, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn3)
    #max pooling                         
    # cnn3 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn3)

    ############################################


    # 合併三個模型的輸出向量
    cnn = tf.keras.layers.concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = tf.keras.layers.Flatten()(cnn)

    # drop1 = tf.keras.layers.Dropout(0.5)(flat)

    dense = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal')(flat) # ,
                                # kernel_regularizer=regularizers.L1(0.1))(flat)
    
    # drop1 = tf.keras.layers.Dropout(0.5)(dense)

    dense = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal')(dense) # ,
                                # kernel_regularizer=regularizers.L1(0.1))(dense)

    dense = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal')(dense) # ,
                                # kernel_regularizer=regularizers.L1(0.05))(dense)

    # drop1 = tf.keras.layers.Dropout(0.5)(dense)

    dense = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal')(dense) # ,
                                # kernel_regularizer=regularizers.L1(0.05))(dense)

    # drop1 = tf.keras.layers.Dropout(0.5)(dense)
    dense = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(dense) # ,
                                # kernel_regularizer=regularizers.L1(0.05))(dense)

    dense = tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_normal')(dense)

    cnn_output = tf.keras.Model(inputs=cnn_input, outputs= dense)


    # 構建ANN模型
    ann_input = tf.keras.layers.Input(shape=(X_train_ann.shape[1],), dtype='float64')

    dense = tf.keras.layers.Dense(128, activation='linear', kernel_initializer='he_normal',
                                    # kernel_regularizer=regularizers.L1(0.1),
                                    activity_regularizer=regularizers.L2(0.0004))(ann_input)
    # drop1 = tf.keras.layers.Dropout(0.5)(dense)

    # bn = tf.keras.layers.BatchNormalization()(drop1)
    dense = tf.keras.layers.Dense(128, activation='linear', kernel_initializer='he_normal',
                                    # kernel_regularizer=regularizers.L1(0.01),
                                    activity_regularizer=regularizers.L2(0.0004))(dense)
    drop1 = tf.keras.layers.Dropout(0.5)(dense)

    dense = tf.keras.layers.Dense(64, activation='linear', kernel_initializer='he_normal',
                                # kernel_regularizer=regularizers.L1(0.01),
                                activity_regularizer=regularizers.L2(0.0004),)(drop1)
    # drop1 = tf.keras.layers.Dropout(0.5)(dense)

    dense = tf.keras.layers.Dense(64, activation='linear', kernel_initializer='he_normal',
                                # kernel_regularizer=regularizers.L1(0.01),
                                activity_regularizer=regularizers.L2(0.0004),)(dense)
    # drop1 = tf.keras.layers.Dropout(0.8)(dense)

    # bn = tf.keras.layers.BatchNormalization()(drop1)
    # dense = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal',
    #                             kernel_regularizer=regularizers.L1(0.01),
    #                             activity_regularizer=regularizers.L2(0.01),)(bn)
    # dense = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(dense)

    ann_output = tf.keras.Model(inputs=ann_input, outputs=dense)

    # combine
    combined = layers.concatenate([cnn_output.output, ann_output.output])

    # bn = tf.keras.layers.BatchNormalization()(combined)
    # dense = tf.keras.layers.Dense(128, activation="relu",kernel_regularizer=regularizers.L1(0.05),)(bn)
    # drop1 = tf.keras.layers.Dropout(0.5)(dense)
    # dense = tf.keras.layers.Dense(128, activation="relu")(drop1)
    # drop1 = tf.keras.layers.Dropout(0.5)(dense)
    dense = tf.keras.layers.Dense(64, activation="relu")(combined) # ,kernel_regularizer=regularizers.L1(0.05),)(bn)
    drop1 = tf.keras.layers.Dropout(0.5)(dense)
    dense = tf.keras.layers.Dense(64, activation="relu")(drop1) #,kernel_regularizer=regularizers.L1(0.05),)(drop1)
    drop1 = tf.keras.layers.Dropout(0.5)(dense)
    # dense = tf.keras.layers.Dense(64, activation="relu")(drop1)
    # drop1 = tf.keras.layers.Dropout(0.2)(dense)
    dense = tf.keras.layers.Dense(32, activation="relu")(drop1) #,kernel_regularizer=regularizers.L1(0.05),)(drop1)
    dense = tf.keras.layers.Dense(1, activation="relu")(dense)

    model = tf.keras.Model(inputs=[cnn_output.input, ann_output.input], outputs=dense)


    # print(model.summary())

    model.compile(loss='mse', optimizer=tf.keras.optimizers.experimental.AdamW(learning_rate=0.03), 
                            metrics=[tf.keras.metrics.MeanSquaredError()])
    print(model.summary())
    

    # batch_size, epochs
    batch_size = 32 
    epochs = 20
    history = model.fit(x = ([X_train_cnn, X_train_ann]), 
                        y = y_train, 
                        validation_data=([X_valid_cnn, X_valid_ann], y_valid), 
                        batch_size = batch_size,
                        epochs = epochs,
                        verbose = 1)


    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc= 'lower right')
    plt.show()



In [24]:
model_(X_train_cnn, X_valid_cnn, X_train_ann, X_valid_ann, y_train, y_valid, embedding_matrix)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 99)]         0           []                               
                                                                                                  
 dense_6 (Dense)                (None, 128)          12800       ['input_3[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 128)          0           ['dense_6[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 128)         512         ['dropout[0][0]']                
 alization)                                                                                 

KeyboardInterrupt: 